In [1]:
import numpy as np
import pandas as pd
import collections, functools, operator

import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#import the table that contains the information about the clusters
viphogs = pd.read_csv("protein.clusters50.onlyClusters.greaterThan2.csv",sep=',',header=None)
viphogs.columns = ["Protein&RegionID","Accession","SourceProtId","SourceProtLength","SourceProtStart","SourceProtEnd","ClusterId","J"]
del(viphogs['J'])
viphogs.head()

,Protein&RegionID,Accession,SourceProtId,SourceProtLength,SourceProtStart,SourceProtEnd,ClusterId
0,253683359@0_150@57_106,AB008336.1,253683359@0_150@57_106,49,1,49,CLS00001
1,2828598@0_120@30_75,AF023424.1,2828598@0_120@30_75,45,1,45,CLS00001
2,2828600@0_122@26_79,AF023425.1,2828600@0_122@26_79,53,1,53,CLS00001
3,4883489@68_228@68_115,AF070476.1,4883489@68_228@68_115,47,1,47,CLS00001
4,4093141@0_150@56_117,AF081782.1,4093141@0_150@56_117,61,1,61,CLS00001


In [3]:
#How many ViPhOGs are?
viphogs["ClusterId"].unique().shape[0]

31150

In [4]:
#How many regions are per viphog
print viphogs.groupby(["ClusterId"]).count()["Protein&RegionID"].median()
print viphogs.groupby(["ClusterId"]).count()["Protein&RegionID"].describe()

5.0
count    31150.000000
mean        17.624655
std         66.334128
min          3.000000
25%          3.000000
50%          5.000000
75%         11.000000
max       3440.000000
Name: Protein&RegionID, dtype: float64


In [6]:
#How many genomes are per viphog?

#1.get pairs viphogs - genomes
pvg = viphogs[["ClusterId","Accession"]].copy() #pvg means paired viphogs-genomes
#2.remove duplicated pairs. That is, if a viphog has several times a genome, keep only one
pvg.drop_duplicates(inplace=True)
#3.count how many genomes are per viphog
print pvg.groupby(["ClusterId"]).count()["Accession"].median()
print pvg.groupby(["ClusterId"]).count()["Accession"].describe()

5.0
count    31150.000000
mean        16.619422
std         52.735551
min          3.000000
25%          3.000000
50%          5.000000
75%         11.000000
max       2139.000000
Name: Accession, dtype: float64


In [7]:
#Which is the viphog with more regions? Is the same viphog after remove the redundancy of genomes?
#1. viphog with more regions
maxViPhog = viphogs.groupby(["ClusterId"]).count()["Protein&RegionID"].max()
print viphogs.groupby(["ClusterId"]).count()["Protein&RegionID"][viphogs.groupby(["ClusterId"]).count()["Protein&RegionID"]==maxViPhog]

#2 viphog with more regions after remove genomes redundancy
maxViPhog = pvg.groupby(["ClusterId"]).count()["Accession"].max()
print pvg.groupby(["ClusterId"]).count()["Accession"][pvg.groupby(["ClusterId"]).count()["Accession"]==maxViPhog]

ClusterId
CLS00937    3440
Name: Protein&RegionID, dtype: int64
ClusterId
CLS00105    2139
Name: Accession, dtype: int64


In [8]:
# Top 10 viphogs with more regions
viphogs.groupby(["ClusterId"]).count()["Protein&RegionID"].sort_values(ascending=False).head(10)

ClusterId
CLS00937    3440
CLS01191    3095
CLS00105    2419
CLS00655    2151
CLS00332    1934
CLS01992    1915
CLS00581    1894
CLS01184    1685
CLS00588    1654
CLS00020    1585
Name: Protein&RegionID, dtype: int64

In [9]:
# Top 10 viphogs with more regions after remove genomes redundancy
pvg.groupby(["ClusterId"]).count()["Accession"].sort_values(ascending=False).head(10)

ClusterId
CLS00105    2139
CLS01191    1392
CLS00588    1273
CLS00581    1246
CLS00167    1198
CLS00937    1180
CLS00170    1163
CLS00169    1163
CLS00175    1160
CLS00168    1159
Name: Accession, dtype: int64

In [10]:
#How many genomes have the viphog with more regions?
viphogs[viphogs["ClusterId"]=="CLS00937"]["Accession"].unique().shape

(1180,)

In [11]:
# Make a table of the Top viphogs (viphogs with more than 100 regions)
topViphogs = viphogs.groupby(["ClusterId"]).count()["Protein&RegionID"].sort_values(ascending=False)

In [12]:
#get the top viphogs
topViphogsDF = pd.DataFrame(topViphogs[topViphogs>100])
topViphogsDF.reset_index(inplace=True)
topViphogsDF.columns = ["ClusterID","NumRegions"]

In [13]:
#get the number of genomes per viphogs
topViphogsDF["NumGenomes"] = pvg.groupby(["ClusterId"]).count()["Accession"].sort_values(ascending=False)[topViphogsDF["ClusterID"]].values

In [198]:
#import regions annotation description
regionsAnnotation = pd.read_csv("../../../clustersAnnotation.forGuille.csv",header=0,sep='\t')
#
#get annotation of each cluster
funcAnnotation = []
for cluster in topViphogsDF["ClusterID"]:
    annotation = []
    x = regionsAnnotation[regionsAnnotation["Cluster"]==cluster]["Description"].value_counts()
    for i,v in x.iteritems():
        annotation.append((i,v))
    funcAnnotation.append(annotation)
#
topViphogsDF["functionalAnnotation"] = funcAnnotation

In [202]:
#Remove undesired chars (- .)from the annotations
#those chars make the same word look different (eg. "domain" "--domain--" "domain.")
def removeChars(a):
    v = a[1]
    a = a[0]
    a = a.replace('-',' ').replace('.',' ')
    return (a,v)

newAnnotations = []
for annotationSet in topViphogsDF["functionalAnnotation"]:
    editedAnnotation = []
    for a in annotationSet:
        editedAnnotation.append(removeChars(a))
    newAnnotations.append(editedAnnotation)
topViphogsDF["functionalAnnotation"] = newAnnotations

In [271]:
def getNodes(avTuple):
    a = avTuple[0]
    v = avTuple[1]
    return(dict(zip(a.split(),[v]*len(a.split()))))

def getEdges(avTuple):
    a = avTuple[0].split()
    edges = []
    i = 1
    while i < len(a):
        edges.append((a[i-1],a[i]))
        i += 1
    edges = dict(zip(edges,[1]*len(edges)))
    return edges

newAnnotations = []
for annotationSet in topViphogsDF["functionalAnnotation"]:
    try:
        #Get all possible words in each annotation. Each word will be a node
        nodes = []
        for a in annotationSet:
            nodes.append(getNodes(a))
        #sum all nodes that have the same key
        nodes = dict(functools.reduce(operator.add,map(collections.Counter,nodes)))
        ###
        #Get all edges. An edge is formed between two contiguous words.
        edges = []
        for a in annotationSet:
            edges.append(getEdges(a))
        #sum all edges that have the same source,target
        edges = dict(functools.reduce(operator.add,map(collections.Counter,edges)))
        ###
        #Store those nodes in a dataframe
        st_values = pd.DataFrame.from_dict(nodes,orient='index').reset_index()
        st_values.columns = ["st","value"]
        ###
        #Make a directed graph with all nodes and edges
        dg = nx.DiGraph()
        dg.add_weighted_edges_from([(x[0],x[1],edges[x]) for x in edges])
        ###
        #Get a subgraph that includes all predecessors and successors of the top 10 most mentioned words
        subgraphNodes = []
        topNodes = st_values.sort_values(by="value",ascending=False).head(10)["st"]
        for x in topNodes:
            subgraphNodes += list(dg.successors(x))
            subgraphNodes += list(dg.predecessors(x))
        subgraphNodes += list(topNodes)
        subgraphNodes = list(set(subgraphNodes))
        subdg = dg.subgraph(subgraphNodes)
        ###
        #Get the longest path from each source (sentence starter) node
        sink_nodes = [node for node, outdegree in subdg.out_degree(subdg.nodes()).items() if outdegree == 0]
        source_nodes = [node for node, indegree in subdg.in_degree(subdg.nodes()).items() if indegree == 0]
        allLongestPaths = []
        for source in source_nodes:
            longest = 0
            p = ""
            for sink in sink_nodes:
                for path in nx.all_simple_paths(subdg,source=source,target=sink):
                    if len(path)>longest:
                        p = path
                        longest = len(path)
            allLongestPaths.append(p)
        ###
        #Keep only the longest path from all paths found in the previous step.
        longest = 0
        p = ""
        for path in allLongestPaths:
            if len(path) > longest:
                p = path
                longest = len(p)
        ###
        #Write the longest path as the final annotation
        finalAnnotation = ""
        for a in p:
            finalAnnotation += a+"("+str(nodes[a])+") "
        newAnnotations.append(finalAnnotation)
    except:
        #do not work mostly for vFam annotations
        #print annotationSet
        newAnnotations.append("NA")

[('vFam_4709_104_146', 1), ('vFam_6599_31_69', 1), ('vFam_2924_194_242', 1), ('vFam_2924_202_241 vFam_959_301_339', 1)]
[]
[('BRO family, N terminal domain BRO family, N terminal domain', 513), ('BRO family, N terminal domain', 109), ('BRO family, N terminal domain BRO family, N terminal domain  ', 3), ('vFam_496_280_357 vFam_5762_20_118 vFam_388_65_143', 1), ('vFam_650_10_75', 1), ('BRO family, N terminal domain  ', 1)]
[('vFam_3621_206_256', 601), ('vFam_3621_206_254', 1), ('vFam_3621_206_256 vFam_4127_251_279', 1), ('vFam_3621_206_256 vFam_4127_251_278', 1), ('vFam_3621_209_256', 1)]
[('    RNA helicase', 139), ('  RNA helicase', 104), ('  RNA helicase  ', 76), ('vFam_103_93_158', 35), ('vFam_103_94_158', 17), ('   ', 17), ('vFam_103_93_159', 12), (' ', 12), ('vFam_103_94_153', 11), ('    RNA helicase  ', 8), ('RNA helicase', 8), ('vFam_103_94_160', 7), ('vFam_103_95_158', 5), ('vFam_103_92_158', 5), ('vFam_103_95_156', 4), ('vFam_103_94_148', 4), ('RNA helicase  ', 4), ('vFam_103_9

In [273]:
topViphogsDF["FA"] = newAnnotations

In [287]:
#import taxonomic annotation file
genomesTaxAnnotation = pd.read_csv("/home/leonardo/Documents/JaimeLeonardoMorenoGallegoUniandes/JuanSebastian/taxonomicAnnotationGenomesWithVOGs.csv",
                                   header=0,sep=',')
#append taxonomic annotation to the viphogs file
getAcc = lambda x :x.split('.')[0]
viphogs["Acc"] = viphogs["Accession"].apply(getAcc)
viphogsTaxAnnotation = viphogs.merge(right=genomesTaxAnnotation,left_on="Acc",right_on="Accession",how="inner")

#get annotation of each cluster
taxAnnotation = []
for cluster in topViphogsDF["ClusterID"]:
    annotation = []
    x = viphogsTaxAnnotation[viphogsTaxAnnotation["ClusterId"]==cluster]["Family"].value_counts()
    for i,v in x.iteritems():
        annotation.append((i,v))
    taxAnnotation.append(annotation)
#
topViphogsDF["taxonomicAnnotation"] = taxAnnotation

In [288]:
#Save final table
topViphogsDF.to_csv("topViphogs.moreThan100regions.v2.csv",header=True,index=False,sep='\t')

In [14]:
#How many viphogs are made of only phages, only eukaryotic viruses or both?

#1.Include host information

#1.1.Remove version from the accession
removeVersion = lambda x:x.split('.')[0]
viphogs["Acc"] = viphogs["Accession"].apply(removeVersion)
#1.2.Import table that includes host info
aux = pd.read_csv("../../../4_FinalGenomes/summaryFinalGenomes.csv",sep='\t')
#1.3.Merge viphgos table and aux table
viphogs2 = viphogs.merge(how="inner",right=aux,left_on="Acc",right_on="Accession")

#2.How many viphogs are made of only viruses, only phages, both?
pvh = viphogs2[["ClusterId","Host"]].copy() #pvh means paired viphog-host
pvh.drop_duplicates(inplace=True)

pvhViruses = set(pvh[pvh["Host"]=="Eukaryote"]["ClusterId"].values)
print len(pvhViruses)
pvhPhages = set(pvh[pvh["Host"]=="Prokaryote"]["ClusterId"].values)
print len(pvhPhages)

print len(pvhViruses - pvhPhages)
print len(pvhPhages - pvhViruses)
print len(pvhViruses & pvhPhages)

21050
16404
14746
10100
6304


In [15]:
#What is the genomes-regions ratio?
(viphogs.groupby(["ClusterId"]).count()["Protein&RegionID"]/pvg.groupby(["ClusterId"]).count()["Accession"]).describe()

count    31150.000000
mean         1.008142
std          0.088604
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          9.161850
dtype: float64

In [16]:
#What is the number of genomes that do not contribute to any ViPhOG?
aux2 = aux[aux["Representative"]==True].copy()
allGenomes = set(aux2["Accession"].values)
viphogsGenomes = set(viphogs["Acc"].unique())
print len(allGenomes - viphogsGenomes)

958


In [88]:
nonViphogs.to_csv("nonViphogs.csv",sep="\t",index=False)

In [22]:
#Explore if the regions in viphogs were proteins, regions or domains. 
regions = viphogs[["Protein&RegionID","Accession"]].copy()
getRegionType = lambda x : x.count("@")
regions["RegionType"] = regions["Protein&RegionID"].apply(getRegionType)
#import regions annotation description
regionsAnnotation = pd.read_csv("../../../clustersAnnotation.forGuille.csv",header=0,sep='\t')
#combine both tables to confirm that contain the same ammount of clusters and regions
regions = pd.concat([regions,regionsAnnotation],axis=1)
#combine regiontype annotation and analysistype annotation
aux = []
for index,row in regions.iterrows():
    aux.append(str(row["RegionType"])+"|"+str(row["Analysis"]))
regions["rac"] = aux #rac := region analysis combined

In [23]:
regions.head()

,Protein&RegionID,Accession,RegionType,Region,Cluster,Analysis,Description,rac
0,253683359@0_150@57_106,AB008336.1,2,AB008336.1|253683359@0_150@57_106,CLS00001,NaN,NaN,2|nan
1,2828598@0_120@30_75,AF023424.1,2,AF023424.1|2828598@0_120@30_75,CLS00001,vFam,vFam_3710_9_59,2|vFam
2,2828600@0_122@26_79,AF023425.1,2,AF023425.1|2828600@0_122@26_79,CLS00001,vFam,vFam_3710_5_63,2|vFam
3,4883489@68_228@68_115,AF070476.1,2,AF070476.1|4883489@68_228@68_115,CLS00001,vFam,vFam_3710_8_60,2|vFam
4,4093141@0_150@56_117,AF081782.1,2,AF081782.1|4093141@0_150@56_117,CLS00001,vFam,vFam_3710_6_74,2|vFam


In [31]:
regions["rac"].unique()

array(['2|nan', '2|vFam', '1|IPS', '1|nan', '0|nan'], dtype=object)

In [30]:
newrac = {"0|nan":"0|nan","1|nan":"1|nan","2|nan":"2|nan","2|vFam":"2|vFam"}
def getNewRac(x):
    if x in newrac:
        return newrac[x]
    else:
        return "1|IPS" #IPS := InterProScan
regions["rac"] = regions["rac"].apply(getNewRac)

In [32]:
test = regions[["Cluster","rac"]].copy()
test.drop_duplicates(inplace=True)

crDict = {} #cr := clusters and regions
for cluster in test["Cluster"].unique():
    crDict[cluster] = list(test[test["Cluster"]==cluster]["rac"])

In [33]:
#Copy cluster so time is not expended getting this dictionary every time
crDictCopy = {}
for cluster in crDict:
    v = crDict[cluster] 
    #1|nan and 2|nan are regions without annotation, so they are the same. Therefore there value is changed to r|nan
    if '1|nan' in v:
        i = v.index('1|nan')
        v[i] = 'r|nan'
    if '2|nan' in v:
        i = v.index('2|nan')
        v[i] = 'r|nan'
    #Dereplicate r|nan values in the list
    v = list(set(v))
    crDictCopy[cluster] = v
    #Sort list as order doesn't matter. 
    crDictCopy[cluster].sort()
    #Group all annotation ways of a cluster in a single string
    v = crDictCopy[cluster]
    v = " & ".join(v)
    crDictCopy[cluster] = v
#Save dict as DataFrame
newTest = pd.DataFrame()
#newTest.columns = ["Cluster","rac"]
newTest["Cluster"] = crDictCopy.keys()
newTest["rac"] = crDictCopy.values()

In [35]:
newTest['rac'].value_counts()

0|nan                             9953
r|nan                             8103
1|IPS                             5758
0|nan & r|nan                     2309
2|vFam                            2081
2|vFam & r|nan                    1057
0|nan & 1|IPS                      578
1|IPS & r|nan                      562
0|nan & 1|IPS & r|nan              271
1|IPS & 2|vFam                     184
1|IPS & 2|vFam & r|nan             132
0|nan & 2|vFam & r|nan              90
0|nan & 1|IPS & 2|vFam & r|nan      44
0|nan & 2|vFam                      18
0|nan & 1|IPS & 2|vFam              10
Name: rac, dtype: int64

In [38]:
newTest['rac'].value_counts().to_csv("viphogsComposition.csv",index=True,sep='\t',header=False)